##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 평가 마이그레이션하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/evaluator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/evaluator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/evaluator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/evaluator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>
</td>
</table>

평가는 모델 측정과 벤치마킹에 있어 중요한 부분을 차지합니다.

이 가이드는 TensorFlow 1에서 TensorFlow 2로 평가기 작업을 마이그레이션하는 방법을 보여줍니다. Tensorflow 1에서 이 기능은 API가 분산 실행될 때 `tf.estimator.train_and_evaluate`에 의해 구현됩니다. Tensorflow 2에서는 기본 `tf.keras.utils.SidecarEvaluator` 또는 평가기 작업의 사용자 정의 평가 루프를 사용할 수 있습니다.

TensorFlow 1(`tf.estimator.Estimator.evaluate`)과 TensorFlow 2(`Model.fit(..., validation_data=(...))` 또는 `Model.evaluate`) 모두에는 간단한 일련의 평가 옵션이 있습니다. 작업자가 훈련과 평가 사이를 전환하지 않는 경우에는 평가기 작업을 사용하는 것이 좋고 평가를 배포하려는 경우에는 `Model.fit`의 내장 평가를 사용하는 것이 좋습니다.


## 설치하기

In [ ]:
import tensorflow.compat.v1 as tf1
import tensorflow as tf
import numpy as np
import tempfile
import time
import os

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## TensorFlow 1: tf.estimator.train_and_evaluate를 사용하여 평가하기

TensorFlow 1에서는 `tf.estimator.train_and_evaluate`를 사용하여 estimator를 평가하도록 `tf.estimator`를 구성할 수 있습니다.

이 예제에서는 먼저 `tf.estimator.Estimator`를 정의하고 훈련 및 평가 사양을 지정합니다.

In [ ]:
feature_columns = [tf1.feature_column.numeric_column("x", shape=[28, 28])]

classifier = tf1.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf1.train.AdamOptimizer(0.001),
    n_classes=10,
    dropout=0.2
)

train_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train.astype(np.int32),
    num_epochs=10,
    batch_size=50,
    shuffle=True,
)

test_input_fn = tf1.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test.astype(np.int32),
    num_epochs=10,
    shuffle=False
)

train_spec = tf1.estimator.TrainSpec(input_fn=train_input_fn, max_steps=10)
eval_spec = tf1.estimator.EvalSpec(input_fn=test_input_fn,
                                   steps=10,
                                   throttle_secs=0)

그런 다음 모델을 훈련하고 평가합니다. 이 노트북에서 평가가 로컬 실행으로 제한되고 훈련과 평가 사이에 번갈아가며 실행되기 때문에 훈련 간에 동기적으로 실행됩니다. 그러나 estimator를 분산적으로 사용하는 경우 평가기는 전용 평가기 작업으로 실행됩니다. 자세한 정보는 [분산 훈련용 마이그레이션 가이드](https://www.tensorflow.org/guide/migrate/multi_worker_cpu_gpu_training)를 확인하세요.

In [ ]:
tf1.estimator.train_and_evaluate(estimator=classifier,
                                train_spec=train_spec,
                                eval_spec=eval_spec)

## TensorFlow 2: Keras 모델 평가하기

TensorFlow 2에서 훈련에 Keras `Model.fit` API를 사용하는 경우 `tf.keras.utils.SidecarEvaluator`를 사용하여 모델을 평가할 수 있습니다. 이 가이드에 표시되지 않은 평가 메트릭을 TensorBoard에서 시각화할 수도 있습니다.

이를 보여주기 위해 먼저 모델을 정의하고 훈련해 보겠습니다.


In [ ]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
  ])

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model = create_model()
model.compile(optimizer='adam',
              loss=loss,
              metrics=['accuracy'],
              steps_per_execution=10,
              run_eagerly=True)

log_dir = tempfile.mkdtemp()
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(log_dir, 'ckpt-{epoch}'),
    save_weights_only=True)

model.fit(x=x_train,
          y=y_train,
          epochs=1,
          callbacks=[model_checkpoint])

그런 다음 `tf.keras.utils.SidecarEvaluator`를 사용하여 모델을 평가합니다. 실제 훈련에서는 별도의 작업을 사용하여 평가를 수행하고 훈련에 사용할 작업자 리소스를 확보하는 것이 좋습니다.

In [ ]:
data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
data = data.batch(64)

tf.keras.utils.SidecarEvaluator(
model=model,
data=data,
checkpoint_dir=log_dir,
max_evaluations=1
).start()

## 다음 단계

- 사이드카 평가에 대해 자세히 알아보려면 `tf.keras.utils.SidecarEvaluator` API 설명서를 읽어보세요.
- Keras에서 훈련과 평가를 번갈아 가며 수행하는 것을 고려하려면 [다른 내장 메소드](https://www.tensorflow.org/guide/keras/train_and_evaluate)를 읽어보는 것을 고려하세요.